Thanks to [Reference](https://towardsdatascience.com/missing-value-imputation-with-python-and-k-nearest-neighbors-308e7abd273d)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-estate-dataset/data.csv')

In [ ]:
df.isnull().sum()

Original dataset has no missing value but I try to create null values and I will show you how I handle the missing values with KNN imputation.

In [ ]:
i1 = np.random.choice(a=df.index, size=40)
i2 = np.random.choice(a=df.index, size=25)

In [ ]:
df.loc[i1, 'INDUS'] = np.nan
df.loc[i2, 'TAX'] = np.nan

In [ ]:
df.isnull().sum()

# KNN imputation

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(df)
df_imputed = pd.DataFrame(imputed, columns=df.columns)

In [ ]:
df_imputed

In [ ]:
df_imputed.isnull().sum()

# Imputer optimization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))

In [ ]:
def optimize_k(data, target):
    errors = []
    for k in range(1, 20, 2):
        imputer = KNNImputer(n_neighbors=k)
        imputed = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed, columns=df.columns)
        
        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})
        
    return errors

In [ ]:
k_errors = optimize_k(data=df, target='MEDV')
k_errors

In [ ]:
k_errors_df = pd.DataFrame(k_errors)
k_errors_df

In [ ]:
import matplotlib.pyplot as plt
plt.plot(k_errors_df['K'],k_errors_df['RMSE'])
plt.show()

When K=17, we have the optimal value!